In [10]:
import pandas as pd
import numpy as np
import json
import ast


def has_hand_zone(payload_line):
    if payload_line.get('type') != 'GREMessageType_GameStateMessage':
        return False

    gsm = payload_line.get('gameStateMessage')
    if not gsm:
        return False

    return any(
        z.get('type') == 'ZoneType_Hand'
        for z in gsm.get('zones', [])
    )

def is_beginning_phase(payload_line):
    if payload_line.get('type') != 'GREMessageType_GameStateMessage':
        return False
    
    gsm = payload_line.get('gameStateMessage')
    if not gsm:
        return False
    
    turn = gsm.get('turnInfo')
    if not turn:
        return False
    
    return turn.get('phase') == 'Phase_Beginning'

# csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test_4mulligan.csv"
csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test_3.csv"
# csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test.csv"
df = pd.read_csv(csv_path)


# # ------------------------------------------------------------

df['payload'] = df['payload'].apply(ast.literal_eval)
df['msgId'] = df['payload'].apply(lambda x: x.get('msgId'))

# df['payload'] = df['payload'].apply(ast.literal_eval)
end_idx = df[df['payload'].apply(is_beginning_phase).values]

final_hand = end_idx.index[0] + 1
df_next = df.iloc[:final_hand]

df_hands = df_next[df_next['payload'].apply(has_hand_zone).values][1:]

# # ------------------------------------------------------------

player_id = df_hands.iloc[0]['player_id']
seatID = df_hands['payload'].iloc[0].get('systemSeatIds')[0]


# # ------------------------------------------------------------

# gameObjectMap = []
# for item in df_hands['payload']:
#     for sub_item in item.get('gameStateMessage').get('gameObjects'):
#         gameObjectMap.append(
#             {
#                 f"{sub_item.get('instanceId')}": sub_item.get('grpId')
#             }
#         )
gameObjectMap = {}
for item in df_hands['payload']:
    for sub_item in item.get('gameStateMessage').get('gameObjects'):
        gameObjectMap[sub_item.get('instanceId')] = sub_item.get('grpId')

# gameObjectMap = [
#     {'instanceId': 244, 'grpId': 93953},
#     {'instanceId': 244, 'grpId': 93953},
#     {'instanceId': 244, 'grpId': 93953}
# ]

# # ------------------------------------------------------------

# zoneId = 31 -> player 1 hand
# zoneId = 35 -> player 2 hand
# zoneId = 30 -> mulligan retured cards

def get_zones (payload_line, player):
    all_zones = payload_line.get('gameStateMessage').get('zones')

    for item in all_zones:
        if player == 'p1' and item.get('zoneId') == 31:
            return item.get('objectInstanceIds')
        if player == 'p2' and item.get('zoneId') == 35:
            return item.get('objectInstanceIds')
        if player == 'limbo' and item.get('zoneId') == 30:
            return item.get('objectInstanceIds', '')


# p1, p2, limbo
df_hands['hand_p1'] = df_hands['payload'].apply(get_zones, args=('p1',))
df_hands['hand_p2'] = df_hands['payload'].apply(get_zones, args=('p2',))
df_hands['hand_limbo'] = df_hands['payload'].apply(get_zones, args=('limbo',))

# # ------------------------------------------------------------

# map hands to grpids
# 
# gameObjectMap = [
#     {'instanceId': 244, 'grpId': 93953}, 
#     {'instanceId': 244, 'grpId': 93953}
# ]
def map_grpid(hand, grpid_map): 
    if hand is None: 
        return None 
    temp = [grpid_map.get(i) for i in hand if i in grpid_map]
    if len(temp) == 0:
        return None
    return temp


df_hands['hand_p1_grpid'] = df_hands['hand_p1'].apply(map_grpid, args=(gameObjectMap,))
df_hands['hand_p2_grpid'] = df_hands['hand_p2'].apply(map_grpid, args=(gameObjectMap,))
df_hands['hand_limbo_grpid'] = df_hands['hand_limbo'].apply(map_grpid, args=(gameObjectMap,))
df_hands

# # ------------------------------------------------------------
# write the hands to disk

# hand_id, match_id will be given to the function
hand_id = 1
match_id = 1

# get the relevant player details (for mulliganCount), and put in a column
def player_details(row):
    players = row['payload'].get('gameStateMessage', {}).get('players', [])

    if row['hand_p1_grpid'] is not None:
        return next(
            (item for item in players if item.get('systemSeatNumber') == 1),
            None
        )
    if row['hand_p2_grpid'] is not None:
        return next(
            (item for item in players if item.get('systemSeatNumber') == 2),
            None
        )
    return None
df_hands['player'] = df_hands.apply(player_details, axis=1)

hands_dict = []
from collections import Counter
last_hand = []
for index, row in df_hands.iterrows():
    if row['hand_p1_grpid'] is not None:
        if row['hand_p1_grpid'] == last_hand:
            hands_dict.pop()
        hands_dict.append({
            'hand_id':hand_id,
            'init_hand': row['hand_p1_grpid'],
            'mulliganCount': row['player'].get('mulliganCount', 0),
            'limbo_hand': row['hand_limbo_grpid'],
            'player_num': row['player'].get('systemSeatNumber')
        })
        last_hand = row['hand_p1_grpid']
    if row['hand_p2_grpid'] is not None:
        if row['hand_p2_grpid'] == last_hand:
            hands_dict.pop()
        hands_dict.append({
            'hand_id':hand_id,
            'init_hand': row['hand_p2_grpid'],
            'mulliganCount': row['player'].get('mulliganCount', 0),
            'limbo_hand': row['hand_limbo_grpid'],
        })
        last_hand = row['hand_p2_grpid']
    hand_id += 1
# print(df_hands['payload'].iloc[0].get('gameStateMessage').get('players'))
# [].get('mulliganCount'))
# df_hands['player'].iloc[1]
from pprint import pprint
pprint(hands_dict)

import duckdb
conn = duckdb.connect("mtga_local.duckdb")  # or ':memory:'
for item in hands_dict:
    went_first = False
    if item.get('player_num') == 1:
        went_first = True

    conn.execute(
        """
        INSERT INTO turn1_hands (hand_id, player_id, match_id, initial_hand, mulliganCount, final_hand, went_first)
        SELECT ?, ?, ?, ?, ?, ?, ?
        """,
        (int(item.get('hand_id')), str(player_id), int(match_id), str(item.get('init_hand')), 
        int(item.get('mulliganCount')), str(item.get('limbo_hand')), str(went_first)
    ))
conn.close()

[{'hand_id': 1,
  'init_hand': [87246, 90793, 90793, 90793, 90793, 91644, 91644],
  'limbo_hand': None,
  'mulliganCount': 0},
 {'hand_id': 2,
  'init_hand': [91654, 87231, 90793, 90793, 90793, 92204, 91619],
  'limbo_hand': None,
  'mulliganCount': 1},
 {'hand_id': 3,
  'init_hand': [90793, 90793, 90793, 91619, 91654, 92204],
  'limbo_hand': [87231],
  'mulliganCount': 1}]


In [11]:
import duckdb

# If you already have a connection
conn = duckdb.connect('mtga_local.duckdb')

# Fetch all rows from the table
rows = conn.execute("SELECT * FROM turn1_hands").fetchall()
conn.close()
from pprint import pprint
pprint(rows)

[(1,
  'V7JT5YS7ANCWRHG35IZNV4PKOY',
  1,
  '[87246, 90793, 90793, 90793, 90793, 91644, 91644]',
  0,
  'None',
  False),
 (2,
  'V7JT5YS7ANCWRHG35IZNV4PKOY',
  1,
  '[91654, 87231, 90793, 90793, 90793, 92204, 91619]',
  1,
  'None',
  False),
 (3,
  'V7JT5YS7ANCWRHG35IZNV4PKOY',
  1,
  '[90793, 90793, 90793, 91619, 91654, 92204]',
  1,
  '[87231]',
  False)]


In [17]:
def get_game_objects(payload_line):
    objs = payload_line.get('gameStateMessage', {}).get('gameObjects', [])
    for item in objs:
        instance_id = item.get('instanceId')
        grp_id = item.get('grpId')
        if instance_id is not None:
            gameObjectMap[instance_id] = grp_id
    # no return needed, we are updating the external dict

# apply for side-effects (ignore returned Series)
df_hands['payload'].apply(get_game_objects)

# now gameObjectMap contains all objects from all payloads
print(gameObjectMap)

{239: 91644, 240: 91644, 241: 90793, 242: 90793, 243: 90793, 244: 90793, 245: 87246, 299: 91619, 300: 92204, 301: 90793, 302: 90793, 303: 90793, 304: 87231, 305: 91654, 359: 87231}


In [88]:
mydict = {
    'key': 'value'
}
def update(mydict):
    mydict.update(
        {'key2': 'val2'}
    )
update(mydict)
mydict

{'key': 'value', 'key2': 'val2'}

In [ ]:

import ast

df['payload'] = df['payload'].apply(ast.literal_eval)

    # grab only the begining hands
# start_idx = df.index[df['payload'].apply(has_hand_zone)].first_valid_index()
mask = df['payload'].apply(has_hand_zone)
start_idx = mask[mask].index[0]
end_idx = df.index[df['payload'].apply(is_beginning_phase)].first_valid_index()

open_hands = df.loc[start_idx:end_idx]
open_hands

In [39]:
import pandas as pd
import json
import ast

def has_hand_zone(payload_line):
    if payload_line.get('type') != 'GREMessageType_GameStateMessage':
        print("No GSM type")
        return False

    gsm = payload_line.get('gameStateMessage')
    if not gsm:
        print("No GSM msg")
        return False

    return any(
        z.get('type') == 'ZoneType_Hand'
        for z in gsm.get('zones', [])
    )

csv_path = "/home/r3gal/develop/mtga_pipeline/cloud/test_4mulligan.csv"
df = pd.read_csv(csv_path)
# df['payload'] = df['payload'].apply(json.loads)
# df = df.drop(0)
df['payload'] = df['payload'].apply(ast.literal_eval)  
# df_one_row = df.iloc[[6]] 
df_new = df[df['payload'].apply(has_hand_zone)]
df_new
pd.set_option('display.max_colwidth', default)
# display(df_one_row['payload'])

No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type
No GSM type


NameError: name 'default' is not defined